In [38]:
import pandas as pd
import MetaTrader5 as mt5
import talib as ta
from datetime import datetime,timedelta
import warnings
import sys
sys.path.insert(1, r'C:\\Users\\Usuário\\Documents\\Github\\Forex-Forecast\\Notebooks')
from bb_setup import setup_1_old,setup_2_old
warnings.filterwarnings("ignore")

In [39]:
if not mt5.initialize(path = r'C:\Program Files\Zero Markets MetaTrader 5 Terminal\terminal64.exe'):
    print("initialize() failed")
    mt5.shutdown()


In [40]:
time_period = 20
var_bb = 2
rate_tp = 2
rate_stop = 1
pontos = 0.0005
spread = 0.000005

ativo = 'EURUSD'
df1 = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0,9999))
df1['data'] = pd.to_datetime(df1['time'], unit='s')
df1['data'] = df1['data'].apply(lambda x: str(x)) 
df = df1[df1['data'] > '2022-01-01']
df = df[['data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})

df_acao = setup_1_old(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)

,Data,Open,High,Low,Close,Vol
0,2022-11-16 07:45:00,1.03725,1.03729,1.03694,1.03719,0
1,2022-11-16 07:50:00,1.03720,1.03722,1.03687,1.03718,0
2,2022-11-16 07:55:00,1.03718,1.03745,1.03666,1.03730,0
3,2022-11-16 08:00:00,1.03728,1.03807,1.03722,1.03722,0
4,2022-11-16 08:05:00,1.03723,1.03737,1.03653,1.03671,0
...,...,...,...,...,...,...
9994,2023-01-06 00:35:00,1.05214,1.05216,1.05210,1.05215,0
9995,2023-01-06 00:40:00,1.05215,1.05215,1.05199,1.05204,0
9996,2023-01-06 00:45:00,1.05203,1.05211,1.05197,1.05209,0
9997,2023-01-06 00:50:00,1.05212,1.05214,1.05184,1.05192,0


In [15]:
timeframe = 5
ativo = f'EURUSD{timeframe}'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(rf'C:\Users\Usuário\Documents\Github\Forex-Forecast\Notebooks\Dados\{ativo}.csv', names = colnames, sep='\t')
df = df[df['Data'] > '2022-06-01']

In [36]:
time_period = 20
var_bb = 2
rate_tp = 2
rate_stop = 1
pontos = 0.0005
spread = 0.000005

timeframe = 15
ativo = f'EURUSD{timeframe}'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(rf'C:\Users\Usuário\Documents\Github\Forex-Forecast\Notebooks\Dados\{ativo}.csv', names = colnames, sep='\t')
df = df[df['Data'] > '2022-06-01']

df_acao = setup_1_old(df,var_bb = var_bb,time_period = 20,pontos = pontos,rate_stop = 1,rate_tp = rate_tp)


In [37]:
qtd_operacoes = df_acao.shape[0]
taxa_acerto = df_acao['resultado_binario'].sum()/qtd_operacoes
operacoes_certas = qtd_operacoes * taxa_acerto
operacoes_erradas = qtd_operacoes - operacoes_certas
pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

pontos_liquido = pontos_bruto - spread * qtd_operacoes

print(f'Timeframe: {timeframe} --pontos: {pontos} --- rate_tp: {rate_tp} -- var_bb: {var_bb}')

print(f'Pontos_liquido: {pontos_liquido} -- Total_operacoes: {qtd_operacoes}')

df_acao['Hora'] = df_acao['Data'].apply(lambda x: x[11:])
df_acao['Hora_h'] = df_acao['Hora'].apply(lambda x: x[:2])
df_acao['Mes'] = df_acao['Data'].apply(lambda x: x[5:7])

df_new_mes = df_acao[['Mes','resultado_binario']]
df_new_hora = df_acao[['Hora_h','resultado_binario']]

def calc_pontos(x):
    qtd_operacoes = x['qtd_operacoes']
    taxa_acerto = x['taxa_acerto']
    operacoes_certas = qtd_operacoes * taxa_acerto
    operacoes_erradas = qtd_operacoes - operacoes_certas
    pontos_bruto =  (operacoes_certas * rate_tp - operacoes_erradas * rate_stop) * pontos

    pontos_liquido = pontos_bruto - spread * qtd_operacoes

    return pontos_liquido

print('Agrupado por Hora')
df_new_hora = df_new_hora.groupby(['Hora_h'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_hora['pontos_liquido'] = df_new_hora.apply(calc_pontos, axis = 1)
print(df_new_hora)

print('Agrupado por Mes')
df_new_mes = df_new_mes.groupby(['Mes'],as_index=False).agg(qtd_operacoes=('resultado_binario', 'count'), taxa_acerto=('resultado_binario', 'mean'))

df_new_mes['pontos_liquido'] = df_new_mes.apply(calc_pontos, axis = 1)
print(df_new_mes)
print('')

Timeframe: 15 --pontos: 0.0005 --- rate_tp: 2 -- var_bb: 2
Pontos_liquido: 0.01073 -- Total_operacoes: 454
Agrupado por Hora
   Hora_h  qtd_operacoes  taxa_acerto  pontos_liquido
0      00             16     0.375000        0.000920
1      01             14     0.500000        0.003430
2      02              6     0.333333       -0.000030
3      03             19     0.315789       -0.000595
4      04             30     0.333333       -0.000150
5      05             40     0.475000        0.008300
6      06             35     0.342857        0.000325
7      07             18     0.277778       -0.001590
8      08             14     0.285714       -0.001070
9      09             17     0.176471       -0.004085
10     10             19     0.421053        0.002405
11     11             29     0.137931       -0.008645
12     12             20     0.150000       -0.005600
13     13             34     0.235294       -0.005170
14     14             13     0.769231        0.008435
15     15  